In [1]:
#import libraries
# This ensures visualizations are plotted inside the notebook
%matplotlib inline
import io
import os              # This provides several system utilities
import pandas as pd    
import seaborn as sns 
import numpy as np
import rtree
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
from cenpy import products
import cenpy
import scipy.stats  as stats # low-level stats & probability
import statsmodels.formula.api as smf # high-level stats
import requests
import contextily as ctx
import rasterio as rio

In [3]:
#reproduce paper data
austria = pd.read_csv('AT_Austria.csv')
austria.head()

,Origin,Destination,Data,Oi,Dj,Dij,Offset,beta,OrigAT11,OrigAT12,...,DestAT12,DestAT13,DestAT21,DestAT22,DestAT31,DestAT32,DestAT33,DestAT34,Oi2007,Dj2007
0,AT11,AT11,0,4016,5146,1.000000e-300,1.000000e-300,1,0,0,...,0,0,0,0,0,0,0,0,4320,5452
1,AT11,AT12,1131,4016,25741,1.030018e+02,1.000000e+00,1,0,0,...,1,0,0,0,0,0,0,0,4320,27169
2,AT11,AT13,1887,4016,26980,8.420467e+01,1.000000e+00,1,0,0,...,0,1,0,0,0,0,0,0,4320,28710
3,AT11,AT21,69,4016,4117,2.208119e+02,1.000000e+00,1,0,0,...,0,0,1,0,0,0,0,0,4320,4354
4,AT11,AT22,738,4016,8634,1.320075e+02,1.000000e+00,1,0,0,...,0,0,0,1,0,0,0,0,4320,9069


In [13]:
austria = austria[austria['Origin']!=austria['Destination']]
flows = austria['Data'].values
Oi = austria['Oi'].values
Dj = austria['Dj'].values
Dij = austria['Dij'].values
Origin = austria['Origin'].values
Destination = austria['Destination'].values

In [14]:
# !pip install git+https://github.com/pysal/spint.git

In [12]:
from spint.gravity import Gravity
from spint.gravity import Production
from spint.gravity import Attraction
from spint.gravity import Doubly

In [15]:
gravity = Gravity(flows,Oi,Dj,Dij,'exp')

In [17]:
print(gravity.params)

[-8.01822841e+00  8.69316127e-01  8.91445153e-01 -6.22938370e-03]


In [18]:
Oi

array([ 4016,  4016,  4016,  4016,  4016,  4016,  4016,  4016, 20080,
       20080, 20080, 20080, 20080, 20080, 20080, 20080, 29142, 29142,
       29142, 29142, 29142, 29142, 29142, 29142,  4897,  4897,  4897,
        4897,  4897,  4897,  4897,  4897,  8487,  8487,  8487,  8487,
        8487,  8487,  8487,  8487, 10638, 10638, 10638, 10638, 10638,
       10638, 10638, 10638,  5790,  5790,  5790,  5790,  5790,  5790,
        5790,  5790,  4341,  4341,  4341,  4341,  4341,  4341,  4341,
        4341,  2184,  2184,  2184,  2184,  2184,  2184,  2184,  2184],
      dtype=int64)

In [19]:
type(Oi)

numpy.ndarray

In [34]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [78]:
class SpIntDataset(Dataset):
    def __init__(self, input_paras,target_flows):
        '''
        '''
        self.in_torch = torch.from_numpy(input_paras).t().view(-1,1,3).float()
        self.out_torch =  torch.from_numpy(target_flows).float().view(-1,1,1)
        
    def __len__(self):
        return len(self.in_torch)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        in_f = self.in_torch[idx,:,:]
        tar_flow = self.out_torch[idx,:,:]
        sample = {'input': in_f, 'output': tar_flow}
        return sample

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(3,8)
        self.fc2 = nn.Linear(8,1)
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()

In [96]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [23]:
flows

array([ 1131,  1887,    69,   738,    98,    31,    43,    19,  1633,
       14055,   416,  1276,  1850,   388,   303,   159,  2301, 20164,
        1080,  1831,  1943,   742,   674,   407,    85,   379,  1597,
        1608,   328,   317,   469,   114,   762,  1110,  2973,  1252,
        1081,   622,   425,   262,   196,  2027,  3498,   346,  1332,
        2144,   821,   274,    49,   378,  1349,   310,   851,  2117,
         630,   106,    87,   424,   978,   490,   670,   577,   546,
         569,    33,   128,   643,   154,   328,   199,   112,   587],
      dtype=int64)

In [63]:
input_features = np.vstack((Oi,Dj,Dij))
in_torch = torch.from_numpy(input_features)
in_torch = in_torch.t()
in_torch = in_torch.view(-1,1,3)
in_torch = in_torch.float()
in_torch[[0]]

tensor([[[ 4016.0000, 25741.0000,   103.0018]]])

In [77]:
in_torch[[1,2,3],:,:]

tensor([[[ 4016.0000, 26980.0000,    84.2047]],

        [[ 4016.0000,  4117.0000,   220.8119]],

        [[ 4016.0000,  8634.0000,   132.0075]]])

In [79]:
out_torch = torch.from_numpy(flows).float()
out_torch = out_torch.view(-1,1,1)


In [80]:
spint_dataset = SpIntDataset(input_features,flows)

In [98]:
dataloader = DataLoader(spint_dataset, batch_size=1,
                        shuffle=True)

In [107]:
#train the network

for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        
        in_f,out_f = data['input'], data['output']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(in_f)
        loss = criterion(outputs, out_f)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: 183780.013
[1,    40] loss: 6673.160
[1,    60] loss: 90042.732
[2,    20] loss: 7811.993
[2,    40] loss: 89647.225
[2,    60] loss: 7013.658
[3,    20] loss: 10820.226
[3,    40] loss: 86850.269
[3,    60] loss: 183856.629
[4,    20] loss: 8365.182
[4,    40] loss: 87098.809
[4,    60] loss: 6390.959
[5,    20] loss: 8453.620
[5,    40] loss: 87417.210
[5,    60] loss: 185760.985
[6,    20] loss: 9138.998
[6,    40] loss: 184438.154
[6,    60] loss: 89308.667
[7,    20] loss: 7481.894
[7,    40] loss: 183383.550
[7,    60] loss: 7414.908
[8,    20] loss: 9033.652
[8,    40] loss: 183808.432
[8,    60] loss: 87717.354
[9,    20] loss: 183652.235
[9,    40] loss: 10136.458
[9,    60] loss: 86548.831
[10,    20] loss: 8227.132
[10,    40] loss: 86494.611
[10,    60] loss: 184591.225
[11,    20] loss: 10209.967
[11,    40] loss: 6054.517
[11,    60] loss: 5772.297
[12,    20] loss: 263104.463
[12,    40] loss: 7646.138
[12,    60] loss: 7472.574
[13,    20] loss: 6513.70

In [101]:
PATH = './austria_ann.pth'
torch.save(net.state_dict(), PATH)

In [2]:
!pip install tb-nightly

  Created wheel for absl-py: filename=absl_py-0.8.1-cp37-none-any.whl size=121171 sha256=73d1a131a385ce359715e9fb3bcff9c6328e680a84f8b260b12999a40a5ed8ff
  Stored in directory: C:\Users\liuli\AppData\Local\pip\Cache\wheels\a7\15\a0\0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
Successfully built absl-py


In [4]:
!pip install future

  Created wheel for future: filename=future-0.18.0-cp37-none-any.whl size=490419 sha256=f9562656e320cfd213c57e0743cb4bd538cdf78541d9d01ec12d33d024793908
  Stored in directory: C:\Users\liuli\AppData\Local\pip\Cache\wheels\2c\02\af\63eadc269fe686aa0aa9c38eee165ad5734cbf8b765cfeedaa
Successfully built future


In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
!pip install tensorboard

In [7]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
Traceback (most recent call last):
  File "c:\programdata\anaconda3\envs\geo\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\geo\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\geo\Scripts\tensorboard.exe\__main__.py", line 9, in <module>
  File "c:\programdata\anaconda3\envs\geo\lib\site-packages\tensorboard\main.py", line 64, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "c:\programdata\anaconda3\envs\geo\lib\site-packages\absl\app.py", line 299, in run
    _run_main(main, args)
  File "c:\programdata\anaconda3\envs\geo\lib\site-packages\absl\app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "c:\programdata\anaconda3\envs\geo\lib\site-packages\tensorboard\program.py", line 220, in main
    server = self._make_server()
  File "c:\programdata\anaco

In [103]:
dataiter = iter(dataloader)
data = dataiter.next()
data

{'input': tensor([[[29142.0000,  4117.0000,   249.9329]]]),
 'output': tensor([[[1080.]]])}

In [104]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [105]:
outputs = net(data['input'])

In [33]:
out_torch.shape

torch.Size([1, 72])

In [46]:
input_test = torch.randn(1, 1, 3)

target_test = torch.randn(1,1,1)
target_test

tensor([[[-0.4147]]])

In [69]:
optimizer.zero_grad()   # zero the gradient buffers


In [70]:

output = net(in_torch)
output


C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[49.7355]],

        [[49.7355]],

        [[68.0691]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[68.0691]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[68.0691]],

        [[49.7355]],

        [[49.7355]],

        [[68.0691]],

        [[68.0691]],

        [[49.7355]],

        [[68.0691]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[49.7355]],

        [[68.0691]],

        [[68.0691]],

        [[49.7355]],

        [[68.0691]],

        [[

In [71]:
loss = criterion(output, out_torch)


In [72]:
loss.backward()
loss.item()

9391175.0

In [73]:
optimizer.step()    # Does the update

In [ ]:
#train the network

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(in_torch)
        loss = criterion(outputs, out_torch)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')